# Delay experiment
how does delay time affect training?
- 2 orthogonal stimuli
- reward: +1 hold +1 final action 0 otherwise

In [1]:
import sys
import os
import itertools
import numpy as np
import torch as tr
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')

import multiprocessing as mp
import concurrent

from utils import *

%load_ext autoreload
%reload_ext autoreload
%autoreload 2


In [2]:
def exp_mp(seed_exp,nseeds):
  """ 
  first argument is seed_exp method
  seed_exp should take one positional argument
    placeholder for iterating over seeds
  """
  with concurrent.futures.ProcessPoolExecutor() as exe:
    data = exe.map(seed_exp, np.arange(nseeds))
  return np.array([i for i in data])
  

In [ ]:
nseeds,neps = 8,10000
muL = [[0,2],[-1,1]]
sigma = 1

dataL = []
for mu in muL:
  print(mu)

  def seed_exp(_seed):
      task = PWMTask(
          stim_set=[(0,1),(1,0)],
          embed_stim='gauss',
          stim_mean=mu,
          stim_var=sigma
        )
      actor = ActorCritic()
      env = Env(actor,task)
      seed_data=np.zeros(neps)
      for ep in range(neps):
        ep_data = env.run_pwm_trial2(update=True)
        seed_data[ep]=ep_data['reward'][-1]
      return seed_data

  # run over multiple seeds
  data = exp_mp(seed_exp,nseeds)
  dataL.append(data)
results = np.array(dataL)

[0, 2]
[-1, 1]


In [ ]:
# todo wrap into plt_hrate(accL,condL)
plt.figure(figsize=(12,5))
for d,res in zip(muL,results):
  M = res.mean(0) 
  S = res.std(0) / np.sqrt(len(res))
  M = M.reshape(-1,50).mean(1)
  plt.plot(M,label=d)
plt.title('N=%iseeds, per variance condition'%nseeds)
plt.ylim(-.05,1.05)
plt.legend()
# plt.savefig('figures/stimvar-aux1')

# todo
- plt_hrate_gsteps(accL) return axarr
- rows: different means; cols: diff sigmas
    - [-1,1],[0,1],[1,8]
    - NB: network size changes weight initial conditions, which might change range/kernell of input layer
- modify exp fun to run seeds in parallel
